<a href="https://colab.research.google.com/github/biomedist/OssData/blob/main/midtask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오픈소스데이터 중간과제물


In [ ]:
import requests
import pandas as pd


url = "http://apis.data.go.kr/1613000/ApHusEnergyUseInfoOfferServiceV2/getWntyAvrgEnergyUseAmountInfoSearchV2"
api_key = '6f8923bf4930eaf1f5dc16e18bd2505221d140710cd395084c84d15bef5791b0'

params = {
    'serviceKey': api_key,
    'searchDate': 'yyyymm',
    'type': 'json'
}


## 데이터 수집
response = requests.get(url, params=params)

## 호출 성공/실패 출력
print(response.json())

{'response': {'body': {'item': {'heat': 0, 'waterHot': 0, 'gas': 0, 'elect': 0, 'waterCool': 0}}, 'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}}}


In [ ]:
response.json()

{'response': {'body': {'item': {'heat': 0,
    'waterHot': 0,
    'gas': 0,
    'elect': 0,
    'waterCool': 0}},
  'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}}}

In [ ]:
import pandas as pd
import requests

## requests로부터 response, body, item 항목 읽기
data = response.json()['response']['body']['item']
print(type(data))
print(len(data))

# 지금껏 읽었던 딕셔너리를 데이터프레임화
df = pd.DataFrame(data, index=[0])
df

<class 'dict'>
5


,heat,waterHot,gas,elect,waterCool
0,0,0,0,0,0


In [ ]:
## JSON 형식 데이터 저장
df.to_json('energy_use.json', indent=4, force_ascii=False)  #indent넣기, 한글을 위해 아스키넣기